In [4]:
# ✅ 1. Importar Librerías
import os
import cv2 as cv
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
import tensorflow as tf

# ✅ 2. Parámetros Globales
IMG_SIZE = (128, 128, 3)
BATCH_SIZE = 32
EPOCHS = 30
LEARNING_RATE = 0.0001  # Reducida para fine-tuning
AUTOTUNE = tf.data.AUTOTUNE

# ✅ 3. Rutas de Datos
ruta_train_csv = r'D:\Archivos de usuarios\Mikel Telo\OneDrive\Documentos\TB-DS-BIO-23.09.24\REPOSITORIOS\Mikel\clip_count\train.csv'
ruta_test_csv = r'D:\Archivos de usuarios\Mikel Telo\OneDrive\Documentos\TB-DS-BIO-23.09.24\REPOSITORIOS\Mikel\clip_count\test.csv'
ruta_train_img = r'D:\Archivos de usuarios\Mikel Telo\OneDrive\Documentos\TB-DS-BIO-23.09.24\REPOSITORIOS\Mikel\clip_count\train'
ruta_test_img = r'D:\Archivos de usuarios\Mikel Telo\OneDrive\Documentos\TB-DS-BIO-23.09.24\REPOSITORIOS\Mikel\clip_count\test'

# ✅ 4. Cargar Datos
df_train = pd.read_csv(ruta_train_csv)
df_test = pd.read_csv(ruta_test_csv)

# ✅ 5. Preprocesamiento de Imágenes
def preprocesar_imagen(img_path):
    img = cv.imread(img_path)
    if img is None:
        raise FileNotFoundError(f"No se pudo cargar la imagen: {img_path}")
    img = cv.cvtColor(img, cv.COLOR_BGR2RGB)
    img = cv.resize(img, (128, 128))
    img = img / 255.0
    return img

def cargar_imagenes_y_etiquetas(df, ruta_img, es_entrenamiento=True):
    imagenes = []
    etiquetas = []
    for _, row in df.iterrows():
        img_path = os.path.join(ruta_img, f"clips-{row['id']}.png")
        img = preprocesar_imagen(img_path)
        imagenes.append(img)
        if es_entrenamiento:
            etiquetas.append(row['clip_count'])
    return np.array(imagenes), (np.array(etiquetas) if es_entrenamiento else None)

X_train, y_train = cargar_imagenes_y_etiquetas(df_train, ruta_train_img, es_entrenamiento=True)
X_test, _ = cargar_imagenes_y_etiquetas(df_test, ruta_test_img, es_entrenamiento=False)

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

# ✅ 6. Crear Modelo Preentrenado con Fine-Tuning
base_model = MobileNetV2(input_shape=IMG_SIZE, include_top=False, weights='imagenet')

# Descongelar las últimas capas
for layer in base_model.layers[-20:]:
    layer.trainable = True

x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)
x = Dense(64, activation='relu')(x)
x = Dropout(0.3)(x)
output = Dense(1, activation='linear')(x)

modelo = Model(inputs=base_model.input, outputs=output)

# Compilar el modelo
modelo.compile(
    optimizer=Adam(learning_rate=LEARNING_RATE),
    loss='mean_squared_error',
    metrics=['mae']
)

modelo.summary()

# ✅ 7. Callbacks
callback_early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
callback_reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-6)
callback_checkpoint = ModelCheckpoint('modelo_mejor.keras', monitor='val_loss', save_best_only=True, mode='min')

# ✅ 8. Entrenar el Modelo
historial = modelo.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    callbacks=[callback_early_stopping, callback_reduce_lr, callback_checkpoint]
)

# ✅ 9. Evaluar y Predecir
loss, mae = modelo.evaluate(X_val, y_val)
print(f"✅ Pérdida en validación: {loss:.4f}, MAE en validación: {mae:.4f}")

y_pred = modelo.predict(X_test)
df_resultados = pd.DataFrame({'id': df_test['id'], 'clip_count': y_pred.flatten()})
df_resultados.to_csv('resultados_clip_count.csv', index=False)

# ✅ 10. Visualizar Resultados
plt.plot(historial.history['loss'], label='Pérdida Entrenamiento')
plt.plot(historial.history['val_loss'], label='Pérdida Validación')
plt.legend()
plt.show()


KeyboardInterrupt: 